# Exercício Prático: Classificação com Múltiplas Métricas (Capítulo 3)
Este notebook contém um exercício prático para aplicar os conceitos de classificação, avaliação de modelos e análise de erros abordados em aula.
Utilizaremos o dataset "Wine", um conjunto de dados público e pequeno com múltiplas classes.


In [1]:
# Execute esta célula para configurar o ambiente do notebook
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier
from sklearn.dummy import DummyClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_score, recall_score, f1_score, precision_recall_curve, roc_curve, roc_auc_score

# --- Carregando o Dataset ---
# O dataset "Wine" possui 178 amostras de vinhos,
# cada uma com 13 características químicas.
# O objetivo é classificar os vinhos em uma de 3 classes (cultivares).

# O que faz: separa o que é entrada e o que é saída do modelo.
# X → as características/ features (ex: teor de álcool, cinzas, cor, etc.)
# y → os rótulos, ou seja, o tipo de vinho correspondente.
# Resumo: X = dados de entrada, y = resposta correta.
wine = load_wine()
X = wine.data
y = wine.target

# Separação dos conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizando os dados para melhor desempenho do SGDClassifier
# O StandardScaler ajusta os dados para que todas as variáveis fiquem com:
# média = 0 (centraliza os dados)
# desvio padrão = 1 (coloca todos mais ou menos no mesmo “tamanho” de variação)
scaler = StandardScaler()

# Aprende (fit) como os dados de treino estão distribuídos (média e desvio padrão).
# Transforma (transform) os dados, aplicando essa normalização.
X_train_scaled = scaler.fit_transform(X_train)

# Aqui só usamos .transform() (sem fit) porque o ajuste já foi feito com o treino.
# Isso garante que o teste seja normalizado usando os mesmos parâmetros (média e desvio) do treino.
X_test_scaled = scaler.transform(X_test)

print("Dataset 'Wine' carregado e pronto.")
print(f"Características do treino: {X_train_scaled.shape}")
print(f"Rótulos do treino: {y_train.shape}")

Dataset 'Wine' carregado e pronto.
Características do treino: (142, 13)
Rótulos do treino: (142,)


## Parte 1: Classificação Binária
Nesta seção, vamos adaptar o problema para uma tarefa de classificação binária, onde o objetivo será apenas identificar se um vinho pertence à classe 0 ou não.

### 1.1 - Adaptar o Dataset para 2 Classes
**Sua tarefa:** Crie as variáveis `y_train_0` e `y_test_0`. Elas devem conter `True` para as amostras que são da classe 0 e `False` para as outras.
Dica: Use uma comparação booleana com `y_train` e `y_test`.



In [2]:
# INSIRA SEU CÓDIGO AQUI
y_train_0 = (y_train == 0)  # True for all 0s, False for all other digits
y_test_0 = (y_test == 0)

print("Primeiros 5 labels do y_train_0:", y_train_0[:5])

Primeiros 5 labels do y_train_0: [False False False False  True]


### 1.2 - Treinar um Classificador Binário e Fazer uma Predição
**Sua tarefa:**
1. Crie e treine uma instância do `SGDClassifier`. Use `random_state=42` para reprodutibilidade.
2. Use o classificador treinado para prever a classe da primeira instância do conjunto de treino (`X_train_scaled[0]`).
3. Imprima a classe prevista e a classe real.

In [3]:
# Criar o modelo
# Aqui você cria o classificador (um tipo de algoritmo que tenta “aprender” a separar as classes).
# random_state=42 serve apenas para que o resultado seja sempre o mesmo quando o código
# for executado novamente.
sgd_clf = SGDClassifier(random_state=42)

# Treinar o modelo
# O método .fit() treina o modelo:
# ele usa os dados de entrada (X_train_scaled) e os rótulos verdadeiros (y_train) para aprender padrões.
sgd_clf.fit(X_train_scaled, y_train_0)

# Fazer uma previsão
# Aqui o modelo prevê a classe da primeira amostra do conjunto de treino (X_train_scaled[0]).
# O resultado (y_pred) será a classe que o modelo acha que é.
# O que é ? Fazemos predict() porque o propósito final de um modelo de aprendizado de máquina é 
# prever resultados. Sem testar com uma previsão, não dá pra saber se ele aprendeu algo de verdade.
# Ou seja, testa se ele aprendeu
y_pred = sgd_clf.predict(X_train_scaled)

# y_pred[0] → o que o modelo previu.
# y_train_0[0] → o que realmente é.
# Assim, você pode comparar e ver se o modelo acertou ou errou.
print("Classe prevista:", y_pred[:10])
print("Classe real:", y_train_0[:10])


Classe prevista: [False False False False  True False False False False  True]
Classe real: [False False False False  True False False False False  True]


## Parte 2: Avaliação de Desempenho (Binário)

### 2.1 - Medir Acurácia com Validação Cruzada
**Sua tarefa:** Use `cross_val_score` para avaliar seu `sgd_clf` com 3 folds (`cv=3`) e a métrica de acurácia. Imprima as pontuações de cada fold.

In [4]:
from sklearn.model_selection import cross_val_score

acuracia_scores = cross_val_score(sgd_clf, X_train_scaled, y_train_0, cv=3, scoring="accuracy")

print("Acurácia em cada fold:", acuracia_scores)

Acurácia em cada fold: [0.95833333 1.         0.9787234 ]


### 2.2 - Baseline com `DummyClassifier`
**Sua tarefa:** Faça o mesmo que no passo anterior, mas usando um `DummyClassifier` com a estratégia "most_frequent". Isso nos dará uma baseline para comparar. O SGD é melhor que o baseline?



In [5]:
# Esse código cria um modelo “burro” (um modelo de comparação) chamado DummyClassifier.
# Ele serve como um modelo básico, para ver se o seu modelo verdadeiro (como o SGDClassifier) 
# realmente está aprendendo algo — ou se só está “chutando”.

# Em resumo:
# Esse código serve para testar se seu modelo real é melhor que um chute.
# Se o DummyClassifier tiver o mesmo desempenho que seu modelo, significa que seu modelo 
# ainda não está aprendendo de verdade.

# Importa o DummyClassifier, que é um classificador de teste (não aprende nada de verdade).
# Ele apenas faz previsões simples, como:
# sempre prever a classe mais comum,
# ou prever aleatoriamente.
from sklearn.dummy import DummyClassifier

# Cria o objeto do tipo DummyClassifier.
# Como não foi passado nenhum parâmetro, ele usará o padrão:
# sempre prever a classe mais frequente dos dados de treino.
dummy_clf = DummyClassifier(strategy="most_frequent")


# “Treina” o modelo (mas na prática, ele só descobre qual é a classe mais comum).
# Ele não aprende padrões como um modelo real faria.
dummy_clf.fit(X_train_scaled, y_train_0)
acuracia_dummy = cross_val_score(dummy_clf, X_train_scaled, y_train_0, cv=3, scoring="accuracy")

print("Acurácia (Dummy) em cada fold:", acuracia_dummy)

Acurácia (Dummy) em cada fold: [0.6875     0.68085106 0.68085106]


### 2.3 - Matriz de Confusão, Precisão, Recall e F1-Score do SGDClassifier (binário)
**Sua tarefa:**
1. Obtenha as previsões para todo o conjunto de treino usando `cross_val_predict`.
2. Calcule e exiba a matriz de confusão.
3. Calcule e imprima a precisão, o recall e o F1-score.



In [6]:
from sklearn.model_selection import cross_val_predict

# Esse código testa o modelo em várias partes diferentes do treino e guarda o que ele previu.
# Isso ajuda a ver se o modelo está realmente aprendendo ou só memorizando.
# A diferença entre cross_val_score e cross_val_predict:
#   Em resumo:
#      cross_val_score → mede o desempenho numérico do modelo.
#      cross_val_predict → mostra o que o modelo previu em cada caso.
#    Ou seja:
#      Use cross_val_score para ver o resultado geral (número).
#      Use cross_val_predict para ver o resultado detalhado (quem acertou e quem errou).
y_train_pred = cross_val_predict(sgd_clf, X_train_scaled, y_train_0, cv=3)

from sklearn.metrics import confusion_matrix

# Matriz de confusão
# Real [[TN, FP]
#      [FN, TP]]
# Verdadeiros Negativos (TN): corretamente classificados
# Falsos Positivos (FP):  incorretamente classificados.
# Falsos Negativos (FN): incorretamente classificados.
# Verdadeiros Positivos (TP): corretamente classificados.
cm = confusion_matrix(y_train_0, y_train_pred)
cm


array([[95,  2],
       [ 1, 44]])

In [7]:
# Calculo da Precisão do modelo

# Diferenca entre cross_val_predict e precision_score:
#   - o cross_val_predict  é uma lista com as previsões do modelo (True/False ou 0/1)
#     para cada amostra do conjunto de treino.
#   - O precision_score mede a precisão do modelo — ou seja:
#         “De todos os casos que o modelo disse ser positivos, quantos ele acertou de verdade?”
from sklearn.metrics import precision_score

precision_score(y_train_0, y_train_pred)  

0.9565217391304348

In [8]:
# Recall
from sklearn.metrics import recall_score
recall_score(y_train_0, y_train_pred)  

0.9777777777777777

In [9]:
# F1 score
from sklearn.metrics import f1_score

f1_score(y_train_0, y_train_pred)

0.967032967032967

## Parte 3: Curvas de Avaliação (Binário)

### 3.1 - Obter Scores de Decisão
**Sua tarefa:** Use `cross_val_predict` novamente, mas desta vez para obter os *scores de decisão* (`decision_function`) em vez das previsões de classe.



In [10]:
# INSIRA SEU CÓDIGO AQUI
y_scores = # ...

print("Scores de decisão (primeiros 5):", y_scores[:5])

SyntaxError: invalid syntax (2570373999.py, line 2)

### 3.2 - Plotar Curvas de Precisão-Recall e ROC
**Sua tarefa:**
1. Use a função `precision_recall_curve` para obter as precisões, recalls e limiares.
2. Plote a curva de Precisão vs. Recall.
3. Use a função `roc_curve` para obter a taxa de falsos positivos (fpr) e a taxa de verdadeiros positivos (tpr).
4. Plote a curva ROC.
5. Calcule e imprima a Área Sob a Curva ROC (AUC).



In [ ]:
# Curva Precisão-Recall

# INSIRA SEU CÓDIGO AQUI
precisions, recalls, thresholds_pr = # ...

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(recalls, precisions, "b-", linewidth=2)
plt.xlabel("Recall")
plt.ylabel("Precisão")
plt.title("Curva Precisão-Recall")
plt.grid(True)

# Curva ROC

# INSIRA SEU CÓDIGO AQUI
fpr, tpr, thresholds_roc = # ...

plt.subplot(1, 2, 2)
plt.plot(fpr, tpr, "b-", linewidth=2, label="SGD")
plt.plot([0, 1], [0, 1], 'k--') # Linha de referência
plt.xlabel("Taxa de Falsos Positivos")
plt.ylabel("Taxa de Verdadeiros Positivos (Recall)")
plt.title("Curva ROC")
plt.grid(True)
plt.show()

# Área Sob a Curva ROC (AUC)
roc_auc = roc_auc_score(y_train_0, y_scores)
print(f"Área Sob a Curva ROC (AUC): {roc_auc:.4f}")
# FIM DO SEU CÓDIGO

## Parte 4: Classificação Multiclasse e Análise de Erros

### 4.1 - Treinar o Classificador Multiclasse
**Sua tarefa:**
1. Treine uma nova instância do `SGDClassifier` (ou use a antiga), mas desta vez usando o `y_train` original, que contém todas as 3 classes.
2. Faça uma predição para a primeira instância do conjunto de treino e imprima a classe prevista e a real.



In [ ]:
# INSIRA SEU CÓDIGO AQUI


### 4.2 - Análise de Erros com Matriz de Confusão
**Sua tarefa:**
1. Use `cross_val_predict` para obter as previsões no conjunto de treino para o modelo multiclasse.
2. Plote a matriz de confusão normalizada (use `ConfusionMatrixDisplay.from_predictions` com `normalize='true'`).
3. Com base na matriz, identifique quais classes o modelo mais confunde.



In [ ]:
y_train_pred_multi = cross_val_predict(sgd_clf_multi, X_train_scaled, y_train, cv=3)

fig, ax = plt.subplots(figsize=(8, 8))

# INSIRA SEU CÓDIGO AQUI

plt.title("Matriz de Confusão Normalizada (por linha)")
plt.show()